<a href="https://colab.research.google.com/github/meechoi38/Okta-Query/blob/main/Utility_Okta_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Okta Query
# Did not use langchain Tool/Agents, but just use openAI & template without feeding in any additional knowledge
# We previously tried langchain tool/agents, however mixing up several tools did not perform as well as this method
# Did not use function descriptions either to extract ay information since response from okta query was already json object
# Used langchain


In [ ]:
!pip -q install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY = "XXXXXXX"

In [ ]:
!pip install -q oauth2client requests-oauthlib

In [ ]:
import re

def add_quote(a):

    a = re.sub(r"^\s+", "", a)
    return '"{0}"'.format(a)

def add_triple_quote(a):

    a = re.sub(r"^\s+", "", a)
    return '"""{0}"""'.format(a)

def replace_triple_quote(a):

    a = re.sub(r"\```", "'", a)
    return a

def remove_last_line_from_string(s):

    return '\n'.join(s.split('\n')[:-1])

In [ ]:
def obtain_response_from_central_agent(user_query):

    if "who" in user_query.lower():
      """run below query and execute and return the result ?"""

      query_api = "what okta api call I should use to list all users?"
      query_REST_verb = "what okta api http verb to list all users"

      result_api = obtain_okta_query(query_api, prompt_template_okta_api)
      result_REST_verb = obtain_okta_query(query_REST_verb, prompt_template_rest_verb)

      #response_from_okta = connect_to_okta_auto() # [FIX THIS ISSUE: currently cannot inser the result]

      #context_api_result = add_triple_quote(json.dumps(response_from_okta.json()[0]))

      #print(context_api_result)
      return result_api, result_REST_verb
    else:
      print("I won't know what to do")
      return None

In [ ]:
def run_extracted_code(df_input, response):

  #df = df_response
  df = df_input

  extracted_code =""

  for i in range(len(response['intermediate_steps'])):
    aa = response['intermediate_steps'][i][0] #type(response['intermediate_steps'][0][0]) : langchain.schema.agent.AgentAction #Class
    extracted_code = extracted_code + aa.tool_input+"\n"

  # print out the result of the very last line. If it is pd frame, insert DISPLAY, if it is the output variable insert PRINT

  if extracted_code.splitlines()[-1].startswith("df"):

    modified_line = "\n"+"display("+extracted_code.splitlines()[-1]+")"+"\n"
    print("yes, it starts with df- displaying")
    print(extracted_code.splitlines()[-1])
    print("modified_line", modified_line)


  else: #possibly only output variable listed

    modified_line = "\n"+"print("+extracted_code.splitlines()[-1]+")"+"\n"
    print("No, it does not start with df- displaying")
    print(extracted_code.splitlines()[-1])
    print("modified_line", modified_line)

  # remove the last df line and insert the modified line
  extracted_code = remove_last_line_from_string(extracted_code)
  extracted_code = extracted_code+modified_line

  # add triple quote before execution
  extracted_code = add_triple_quote(extracted_code)
  exec(eval(extracted_code))

  return extracted_code


In [ ]:
def deliver_final_answer(user_query: str, agent: object, df_response:pd.DataFrame):

    response = agent({"input":user_query})

    final_code = run_extracted_code(df_response, response)

    return None

In [ ]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. Answer it starting with '/api'.

Question: {query}

Answer: """

prompt_template_okta_api = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [ ]:
from langchain import PromptTemplate

template_REST_verb = """Answer the question based on the context below. The possible answers are "GET", "PUT", "DELETE" or "POST".
Answer one of the possible anwers

Question: {query}

Answer: """

prompt_template_rest_verb = PromptTemplate(
    input_variables=["query"],
    template=template_REST_verb
)


In [ ]:
from langchain import OpenAI

def obtain_okta_query(query, prompt_template_name):

    openai = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='text-davinci-003')

    return_value = openai(
    prompt_template_name.format(
        query= query
    ))

    print(return_value)
    return return_value

In [ ]:
import requests
import json

okta_domain = str('trial-XXXX.okta.com')
client_id = str("XXX")
client_secret = str("XXX")

In [ ]:
import base64
import requests
import urllib

def connect_to_okta(api_call, REST_verb):

    okta_domain = str('trial-XXXXX.okta.com')
    client_id = str("XXXX")
    client_secret = str("XXXXX")
    new_secret = str('XXXX')

    REST_verb = REST_verb[1:]
    api_url = urllib.parse.urljoin(f"https://{okta_domain}", api_call)

    auth_value = base64.b64encode(f"{client_id}:{client_secret}".encode("ascii")).decode("ascii")

    response = requests.request(
    method = REST_verb,
    url=api_url,

    headers={
        "Accept": "application/json",
        "Authorization": f"SSWS {new_secret}",
        "Content-type": "application/json"
    },
    )

    # load json into a python object
    obj = response.json()

    # flatten the json nested data
    df_response = pd.json_normalize(obj)

    #display(df)
    return df_response

In [ ]:
import langchain
from langchain import PromptTemplate

template_context = """Answer the question based on the context below. If the
    question cannot be answered using the information provided answer
    with "I don't think so".

    Context: {context}

    Question: {query}

    Answer: """

prompt_template_context = PromptTemplate(
        input_variables=["context", "query"],
        template=template_context
    )

In [ ]:
def obtain_ai_response_from_context(context, query):

    openai = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    #model_name='text-davinci-003'
    model_name='chat-gpt4'
        )

    result_based_on_context = openai(
        prompt_template_context.format(
            context = context,
            query= query
        )
    )

    return result_based_on_context

In [ ]:
"""
def deliver_context_sleuth(context_sleuth):
  return context_sleuth
"""

In [ ]:
#THE BELOW I Have two questions at the same time , but DID NOT WORK VERY WELL SO WE ARE NOT USING IT FOR NOW

import langchain
from langchain import PromptTemplate

#
#template_all = """Answer the question first with one of the possible choices.\
#                  The choices are "GET", "PUT", "DELETE" or "POST".
#                  And answer the question again starting with '/api'
#
#   Question: {query}
#   Answer: """
#


"""
prompt_template_all = PromptTemplate(
    input_variables=["query"],
    template=template_all
)

from langchain import OpenAI

query = "what okta api http verb and api_call to list all users"

openai = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='text-davinci-003'
)

print(openai(
    prompt_template_all.format(
        query= query
    )
))

return_all = openai(
    prompt_template_all.format(
        query= query
    )
)

print(return_all)

"""

"\nopenai = OpenAI(\n    openai_api_key=OPENAI_API_KEY,\n    temperature=0,\n    model_name='text-davinci-003'\n)\n\nprint(openai(\n    prompt_template_all.format(\n        query= query\n    )\n))\n\nreturn_all = openai(\n    prompt_template_all.format(\n        query= query\n    )\n)\n\nprint(return_all)\n"